# Star Formation

In [80]:
import matplotlib.pyplot as plt
import pandas as pd
from scipy.signal import savgol_filter
import numpy as np
import warnings

In [81]:
from auriga.snapshot import Snapshot
from auriga.images import figure_setup
from auriga.settings import Settings
from auriga.cosmology import Cosmology
from auriga.parser import parse

In [82]:
figure_setup()
settings = Settings()
cosmology = Cosmology()

In [83]:
DISC_STD_CIRC = settings.disc_std_circ
DISC_MIN_CIRC = settings.disc_min_circ
COLD_DISC_DELTA_CIRC = settings.cold_disc_delta_circ
BULGE_MAX_SPECIFIC_ENERGY = -0.6
SUFFIX = "_02"

In [84]:
def load_data(simulation: str) -> pd.DataFrame:
    s = Snapshot(simulation, loadonlytype=[0, 1, 2, 3, 4, 5])
    s.tag_particles_by_region(
        disc_std_circ=DISC_STD_CIRC,
        disc_min_circ=DISC_MIN_CIRC,
        cold_disc_delta_circ=COLD_DISC_DELTA_CIRC,
        bulge_max_specific_energy=BULGE_MAX_SPECIFIC_ENERGY)
    s.tag_in_situ_stars()

    is_real_star = (s.type == 4) & (s.stellar_formation_time > 0)
    is_main_obj = (s.halo == s.halo_idx) & (s.subhalo == s.subhalo_idx)

    props = {
        "FormationTime_Gyr": cosmology.expansion_factor_to_time(
            s.stellar_formation_time[is_real_star & is_main_obj]),
        "ComponentTag": s.region_tag[is_real_star & is_main_obj],
        "Mass_Msun": s.mass[is_real_star & is_main_obj],
        "IsInSitu": s.is_in_situ[is_real_star & is_main_obj],
    }

    return pd.DataFrame(props), s.time

In [85]:
def calculate_sfr(simulation: str) -> pd.DataFrame:
    settings = Settings()

    if parse(simulation)[1]:
        raise ValueError("This method uses snapshot 127 as the last snapshot.")

    df, today_time = load_data(f"{simulation}_s127")

    # Calculate SFR by component
    times = np.linspace(0, today_time, 100)  # Gyr
    bin_width = np.diff(times)[0]  # Gyr
    bin_centers = times[1:] - bin_width / 2  # Gyr

    mass, _ = np.histogram(
        df["FormationTime_Gyr"], bins=times, weights=df["Mass_Msun"])

    sfr = {
        "Time_Gyr": bin_centers,
        "SFR_Msun/yr": mass / bin_width / 1E9,
    }

    mass, _ = np.histogram(
        df["FormationTime_Gyr"][df["IsInSitu"] == 1], bins=times,
        weights=df["Mass_Msun"][df["IsInSitu"] == 1])

    sfr["SFR_InSitu_Msun/yr"] = mass / bin_width / 1E9

    # Calculate the SFR by component
    for i, comp in enumerate(settings.components):
        mass, _ = np.histogram(
            df["FormationTime_Gyr"][df["ComponentTag"] == i], bins=times,
            weights=df["Mass_Msun"][df["ComponentTag"] == i])
        sfr[f"SFR_{comp}_Msun/yr"] = mass / bin_width / 1E9
        mass, _ = np.histogram(
            df["FormationTime_Gyr"][
                (df["ComponentTag"] == i) & (df["IsInSitu"] == 1)],
            bins=times,
            weights=df["Mass_Msun"][
                (df["ComponentTag"] == i) & (df["IsInSitu"] == 1)])
        sfr[f"SFR_{comp}_InSitu_Msun/yr"] = mass / bin_width / 1E9

    return pd.DataFrame(sfr)

### Figures

In [86]:
SIMULATION = "au6_or_l4_s127"
# SIMULATION = None

In [87]:
# SAMPLE = [f"au{i}_or_l4" for i in settings.groups["Included"]]
SAMPLE = None

In [88]:
if SIMULATION:
    fig, ax = plt.subplots(figsize=(2.5, 2.5))

    ax.tick_params(which='both', direction="in")
    ax.set_axisbelow(True)
    ax.grid(True, ls='-', lw=0.25, c='silver')

    ax.set_xlim(0, 14)
    ax.set_xticks([2, 4, 6, 8, 10, 12])
    ax.set_xlabel("Time [Gyr]")

    ax.set_ylim(0.1, 20)
    ax.set_yscale("log")
    ax.set_yticks([0.1, 1, 10])
    ax.set_yticklabels([0.1, 1, 10])
    ax.set_ylabel(r"SFR [$\mathrm{M}_\odot \, \mathrm{yr}^{-1}$]")

    label = f"Au{parse(SIMULATION)[0]}"

    sfr = calculate_sfr(SIMULATION)
    ax.plot(sfr["Time_Gyr"], savgol_filter(sfr["SFR_Msun/yr"], 5, 1),
            c='k', lw=1.0, label="Total", zorder=12)
    ax.plot(sfr["Time_Gyr"], savgol_filter(sfr["SFR_InSitu_Msun/yr"], 5, 1),
            c='k', ls=(0, (5, 1)), lw=0.5, label="In Situ", zorder=10)
    for component in settings.components:
        ax.plot(sfr["Time_Gyr"],
                savgol_filter(sfr[f"SFR_{component}_Msun/yr"], 5, 1),
                c=settings.component_colors[component],
                lw=1.0, label=settings.component_labels[component], zorder=12)
        ax.plot(sfr["Time_Gyr"],
                savgol_filter(sfr[f"SFR_{component}_InSitu_Msun/yr"], 5, 1),
                c=settings.component_colors[component], ls=(0, (5, 1)), lw=0.5,
                zorder=10)

    ax.legend(loc="upper right", framealpha=0.0, fontsize=5.0,
              bbox_to_anchor=(0.99, 0.5), ncol=1)

    ax.text(x=0.05, y=0.95, s=r"$\texttt{" + label + "}$",
            size=8.0, transform=ax.transAxes, ha='left', va='top')

    fig.savefig(
        f"../images/sfr_by_region/histogram_{SIMULATION}{SUFFIX}.pdf")
    plt.close(fig)

In [93]:
if SAMPLE:
    fig = plt.figure(figsize=(7, 8))
    gs = fig.add_gridspec(nrows=6, ncols=4, hspace=0.0, wspace=0.0)
    axs = gs.subplots(sharex=True, sharey=True)

    for ax in axs.flatten():
        ax.tick_params(which='both', direction="in")
        if ax == axs[-1, -1]: ax.axis("off")
        ax.set_xlim(0, 14)
        ax.set_ylim(0.1, 20),
        ax.set_xticks([2, 4, 6, 8, 10, 12])
        ax.set_yscale("log")
        ax.set_yticks([0.1, 1, 10])
        ax.set_yticklabels([0.1, 1, 10])
        if ax.get_subplotspec().is_last_row() or ax == axs[-2, -1]:
            ax.set_xlabel("Time [Gyr]")
            ax.tick_params(labelbottom=True)
        if ax.get_subplotspec().is_first_col():
            ax.set_ylabel(r"SFR [$\mathrm{M}_\odot \, \mathrm{yr}^{-1}$]")

    for i, simulation in enumerate(SAMPLE):
        ax = axs.flatten()[i]
        sfr = calculate_sfr(simulation)
        label = f"Au{parse(simulation)[0]}"
        ax.plot(sfr["Time_Gyr"],
                savgol_filter(sfr["SFR_Msun/yr"], 5, 1),
                c='k', lw=1.0, label="Total", zorder=12)
        ax.plot(sfr["Time_Gyr"],
                savgol_filter(sfr["SFR_InSitu_Msun/yr"], 5, 1),
                c='k', ls=(0, (5, 1)), lw=0.5, label="In Situ", zorder=10)
        for component in settings.components:
                ax.plot(sfr["Time_Gyr"],
                        savgol_filter(sfr[f"SFR_{component}_Msun/yr"], 5, 1),
                        c=settings.component_colors[component],
                        lw=1.0, label=settings.component_labels[component],
                        zorder=12)
                ax.plot(sfr["Time_Gyr"],
                        savgol_filter(sfr[f"SFR_{component}_InSitu_Msun/yr"],
                                        5, 1),
                        c=settings.component_colors[component], ls=(0, (5, 1)),
                        lw=0.5, zorder=10)
        ax.text(x=0.95, y=0.95, size=6.0, s=r"$\texttt{" + label + "}$",
                ha='right', va='top', transform=ax.transAxes)

    if ax == axs[0, 0]:
        ax.legend(loc="lower center", framealpha=0.0, fontsize=5.0,
                  bbox_to_anchor=(3.5, -4.9), ncol=2)

    ax.xaxis.label.set_size(8.0)
    ax.yaxis.label.set_size(8.0)

    fig.savefig(f"../images/sfr_by_region/histogram_included{SUFFIX}.pdf")

    plt.close(fig)